## TODO APPLICATION

* We can add todo but with the help of AI.
* We can delete todo but with the help of AI.
* We can update todo but with the help of AI.
* We can read or read particular todo with the help of AI.

### Import Required Things
---

In [60]:
from langgraph.graph import StateGraph, START,END
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from dotenv import load_dotenv
from os import getenv
from typing import List, Annotated
from typing_extensions import TypedDict
from bson import ObjectId

load_dotenv()

DB_URL = getenv("MONOGODB_URL")

### Setup Database
---

In [14]:
from pymongo import MongoClient

client = MongoClient(DB_URL)

db = client["agent"]

collection = db["todo"]

### Create Prompt Template
---

In [15]:
prompt_template = ChatPromptTemplate([
    SystemMessage(content="You are a helpful assistant that take user input based on that call the usable tool."),
    MessagesPlaceholder("messages")
])

### Handler Fuctions
---

In [ ]:
def insert_document(data: dict | List[dict]) -> bool:
    try:
        collection.insert_one(data)
        return True
    except Exception as e:
        print("An execption occureds during inserting the documents:: ".e)
        return False
    
def delete_documents(ids: dict | List[dict]) -> bool:
    try:
        collection.delete_many(ids)
        return True
    except Exception as e:
        print("An execption occureds during deleting the documents:: ".e)
        return False
    
def get_documents(ids: dict | List[dict]) -> list[dict]:
    try:
        collection.find(ids)
        return True
    except Exception as e:
        print("An exception occured during getting the documents:: ",e)
        return False
    
def update_documents(update: list) -> bool:
    try:
        collection.bulk_write(update)
        return True
    except Exception as e:
        print("An exception occured during update the documents:: ",e)
        return False

In [ ]:
from typing import Optional


@tool
def create_todo(message: dict | List[dict]):
    """ create new todo or todos

    Args:
        message : str
    """
    collection.insert_one({"todo" : message})

    return "sucess"

@tool
def delete_todo(todo_id: List[dict] | dict):
    """ delete todo by getting list or id in the form of {"_id" : {"$in" : [ObjectId("id"),...]}}

    Args:
        todo_id : str | List[str]
    """
    pass

@tool
def get_todo(todo_id: Optional[str] = None):
    """ get todos by getting todo ids in the form of dic or list of dict {"id": ObjectId(id)}

    Args:
        todo_id : Optional[str]
    """
    pass

@tool
def update_todo(updated: list):
    """update todo by getting id with update value in the form of like that [UpdateOne({"_id":ObjectId("id)},{"$set":{"todo":"update_message"}})]

    Args:
        todo_id : str        
    """
    pass

tools = [create_todo, delete_todo, get_todo, update_todo]

tools_by_name = {tool.name : tool for tool in tools}

llm = ChatOpenAI(model= "gpt-4o-mini", temperature= 0.3)
llm_with_tools = llm.bind_tools(tools)

### Creating Workflow schema
---

In [ ]:
class State(TypedDict):
    messages: Annotated[list,add_messages]

### Creating Workflow
---

In [ ]:
builder = StateGraph(State)

